In [1]:
import os
os.environ["JAVA_HOME"] = "jdk-11"

In [2]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *

import timeit
start_time = timeit.default_timer()

spark = SparkSession \
      .builder \
      .appName('reading Kafka') \
      .master("local[*]") \
      .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0')\
      .getOrCreate()\

sc = spark._sc

spark.version

:: loading settings :: url = jar:file:/opt/conda/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4ba5fb2e-ef39-4e06-aaa3-31cf187f55f9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central

'3.2.0'

In [3]:
heroes_topic = 'heroes'

df = spark \
  .readStream \
  .format("kafka")\
  .option("kafka.bootstrap.servers", "PLAINTEXT://broker:9093") \
  .option("subscribe", heroes_topic) \
  .load()

In [4]:
schema = StructType([StructField('Name',StringType(),True),
                     StructField('Alignment',StringType(),True),
                     StructField('Intelligence',DecimalType(10, 2),True),
                     StructField('Strength',DecimalType(10, 2),True),
                     StructField('Speed',IntegerType(),True),
                     StructField('Durability',DecimalType(10, 2),True),
                     StructField('Power',DecimalType(10, 2),True),
                     StructField('Combat',DecimalType(10, 2),True),
                     StructField('Total',DecimalType(10, 2),True)])

In [ ]:
query = df\
    .select(f.col("key").cast("string"),
            f.from_json(f.col("value").cast("string"), schema).alias('values'))\
    .selectExpr("key", "values.*")\
    .writeStream \
    .format("console") \
    .option("checkpointLocation", "checkpoint_subscriber") \
    .start()

query.awaitTermination()

21/12/30 17:08:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+----------+---------+------------+--------+-----+----------+-----+------+------+
|key|      Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+----------+---------+------------+--------+-----+----------+-----+------+------+
|476|      Sage|     good|       75.00|   10.00|   12|     14.00|28.00| 56.00|195.00|
|475|Sabretooth|      bad|       55.00|   53.00|   25|     90.00|28.00|100.00|351.00|
+---+----------+---------+------------+--------+-----+----------+-----+------+------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---+-------+---------+------------+--------+-----+----------+-----+------+------+
|key|   Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+-------+---------+------------+--------+-----+----------+-----+------+------+
|477|Sandman|  neutral|       50.00|   75.00|   47|     97.00|62.00| 56.00|387.00|
+---+-------+---------+------------+--------+-----+----------+-----+------+------+



-------------------------------------------
Batch: 2
-------------------------------------------
+---+---------+---------+------------+--------+-----+----------+-----+------+------+
|key|     Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+---------+---------+------------+--------+-----+----------+-----+------+------+
|478|Sasquatch|     good|       75.00|   80.00|   23|     56.00|15.00| 42.00|291.00|
+---+---------+---------+------------+--------+-----+----------+-----+------+------+

-------------------------------------------
Batch: 3
-------------------------------------------
+---+-------------+---------+------------+--------+-----+----------+-----+------+------+
|key|         Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+-------------+---------+------------+--------+-----+----------+-----+------+------+
|479|Savage Dragon|     good|       63.00|   70.00|   54|     85.00|49.00| 72.00|393.00|
+---+-------------+-----

-------------------------------------------
Batch: 4
-------------------------------------------
+---+---------+---------+------------+--------+-----+----------+-----+------+------+
|key|     Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+---------+---------+------------+--------+-----+----------+-----+------+------+
|480|Scarecrow|      bad|       81.00|   10.00|   12|     14.00|48.00| 50.00|215.00|
+---+---------+---------+------------+--------+-----+----------+-----+------+------+

-------------------------------------------
Batch: 5
-------------------------------------------
+---+--------------+---------+------------+--------+-----+----------+-----+------+------+
|key|          Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+--------------+---------+------------+--------+-----+----------+-----+------+------+
|481|Scarlet Spider|     good|       75.00|   53.00|   60|     74.00|39.00| 56.00|357.00|
+---+--------------+

...

-------------------------------------------
Batch: 50
-------------------------------------------
+---+-----+---------+------------+--------+-----+----------+-----+------+------+
|key| Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+-----+---------+------------+--------+-----+----------+-----+------+------+
|539|Sylar|      bad|       75.00|   10.00|   12|     28.00|91.00| 56.00|272.00|
+---+-----+---------+------------+--------+-----+----------+-----+------+------+

-------------------------------------------
Batch: 51
-------------------------------------------
+---+-------+---------+------------+--------+-----+----------+-----+------+------+
|key|   Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+-------+---------+------------+--------+-----+----------+-----+------+------+
|541|Tempest|     good|       38.00|   10.00|   45|     28.00|62.00| 60.00|243.00|
|540|  Synch|     good|       75.00|   67.00|   23|     28.00|74.0

-------------------------------------------
Batch: 53
-------------------------------------------
+---+--------+---------+------------+--------+-----+----------+-----+------+------+
|key|    Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+--------+---------+------------+--------+-----+----------+-----+------+------+
|543|The Cape|     good|       38.00|   10.00|   23|     32.00|22.00| 64.00|189.00|
+---+--------+---------+------------+--------+-----+----------+-----+------+------+

-------------------------------------------
Batch: 54
-------------------------------------------
+---+------------+---------+------------+--------+-----+----------+-----+------+------+
|key|        Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+------------+---------+------------+--------+-----+----------+-----+------+------+
|544|The Comedian|  neutral|       63.00|   14.00|   17|     10.00|12.00| 80.00|196.00|
+---+------------+---------+---

-------------------------------------------
Batch: 55
-------------------------------------------
+---+-----+---------+------------+--------+-----+----------+------+------+------+
|key| Name|Alignment|Intelligence|Strength|Speed|Durability| Power|Combat| Total|
+---+-----+---------+------------+--------+-----+----------+------+------+------+
|546| Thor|     good|       69.00|  100.00|   92|    100.00|100.00| 85.00|546.00|
|545|Thing|     good|       75.00|   84.00|   21|     90.00| 26.00| 80.00|376.00|
+---+-----+---------+------------+--------+-----+----------+------+------+------+

-------------------------------------------
Batch: 56
-------------------------------------------
+---+-----------+---------+------------+--------+-----+----------+------+------+------+
|key|       Name|Alignment|Intelligence|Strength|Speed|Durability| Power|Combat| Total|
+---+-----------+---------+------------+--------+-----+----------+------+------+------+
|547|  Thor Girl|     good|       75.00|   83.0

-------------------------------------------
Batch: 57
-------------------------------------------
+---+---------------+---------+------------+--------+-----+----------+-----+------+-----+
|key|           Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+---+---------------+---------+------------+--------+-----+----------+-----+------+-----+
|550|Thunderbird III|     good|        1.00|    1.00|    1|      1.00| 0.00|  1.00| 5.00|
|549| Thunderbird II|     good|        1.00|    1.00|    1|      1.00| 0.00|  1.00| 5.00|
+---+---------------+---------+------------+--------+-----+----------+-----+------+-----+

-------------------------------------------
Batch: 58
-------------------------------------------
+---+-------------+---------+------------+--------+-----+----------+-----+------+------+
|key|         Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+-------------+---------+------------+--------+-----+----------+-----+------+----

-------------------------------------------
Batch: 61
-------------------------------------------
+---+-----+---------+------------+--------+-----+----------+-----+------+------+
|key| Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+-----+---------+------------+--------+-----+----------+-----+------+------+
|554|Tigra|     good|       63.00|   32.00|   53|     38.00|33.00| 90.00|309.00|
+---+-----+---------+------------+--------+-----+----------+-----+------+------+

-------------------------------------------
Batch: 62
-------------------------------------------
+---+--------+---------+------------+--------+-----+----------+-----+------+------+
|key|    Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat| Total|
+---+--------+---------+------------+--------+-----+----------+-----+------+------+
|555|Tinkerer|      bad|      100.00|   10.00|   23|     14.00|10.00| 14.00|171.00|
|556|   Titan|     good|        1.00|    1.00|    1|      1.00

In [ ]:
query = df\
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .format("console") \
    .option("checkpointLocation", "checkpoint_subscriber") \
    .start()

query.awaitTermination()